# Iris Detection - Data Collection

---


Steps to complete:

1. Take Pictures
2. Partition Data
3. Annotate Labels
4. Data Augmentation (just training data)


---

## 1. Dependencies installed

- labelme
- opencv-python
- matplotlib
- albumentations
- notebook (for juypter notebook)


---

## 1. Take Pictures

Selfies in different positions


---

## 2. Partition Data

Create random train, val and test data set (Proportions: 50:25:25).

Move Images from data folder to their corresponding new folder.


In [52]:
import os
import random
import shutil

In [53]:
# define folder paths
source_folder = "data/images"
train_folder = "data/train/images"
val_folder = "data/val/images"
test_folder = "data/test/images"

In [54]:
# create destination folders if dont exist already
os.makedirs(train_folder, exist_ok=True)
os.makedirs(val_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

In [55]:
# list all available images
image_files = os.listdir(source_folder)

In [56]:
# shuffle images randomly
random.shuffle(image_files)

In [57]:
# calculate number of images per set
total_images = len(image_files)
train_size = int(0.5 * total_images)
val_size = int(0.25 * total_images)
test_size = total_images - train_size - val_size

In [58]:
# check sizes
total_images, train_size, val_size, test_size

(103, 51, 25, 27)

In [59]:
# Move images to train folder
for image_file in image_files[:train_size]:
    shutil.copy(
        os.path.join(source_folder, image_file), os.path.join(
            train_folder, image_file)
    )

# Move images to val folder
for image_file in image_files[train_size: train_size + val_size]:
    shutil.copy(
        os.path.join(source_folder, image_file), os.path.join(
            val_folder, image_file)
    )

# Move images to test folder
for image_file in image_files[train_size + val_size:]:
    shutil.copy(
        os.path.join(source_folder, image_file), os.path.join(
            test_folder, image_file)
    )

---
## 3. Label Data with Labelme
---


## 4. Data Augmentation


In [10]:
# Our data augmentation library
import albumentations as alb
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt

We will use different augmentation operations to create a big training data set.

- Randomly cropping input images to specified shape
- Flipping input images horizontally with probability of 0.5
- Randomly adjusting brightness & contrast of input images with probability of 0.2
- Randomly adjusting gamma (also brightness but different operation) of the imput images with probability of 0.2
- Randomly shifting rgb values of input images with probability of 0.2
- Flipping input images vertically with probability of 0.5
- Taking into the original label position into consideration while augmenting (label augmentation)


In [64]:
size_in_pixels = 460
augmentor = alb.Compose(
    [
        alb.RandomCrop(width=size_in_pixels, height=size_in_pixels),
        alb.HorizontalFlip(p=0.5),
        alb.RandomBrightnessContrast(p=0.4),
        alb.RandomGamma(p=0.4),
        alb.RGBShift(p=0.4),
        alb.VerticalFlip(p=0.5),
    ],
    keypoint_params=alb.KeypointParams(
        format="xy", label_fields=["class_labels"]),
)

In [46]:
# # iterate over all images in train set
# for image in os.listdir(os.path.join('data', 'train', 'images')):
#     # i += 1 # remove later
#     # opens image as array
#     img = cv2.imread(os.path.join('data', 'train', 'images', image))

#     # initiate classes (eyes visible or not, left and right)
#     classes = [0, 0]

#     # initiate coordiates of eyes if visible if not 0 (x-left, y-left, x-right, y-right)
#     coords = [0, 0, 0, 0]

#     # get corresponding label to current image (they have same file name)
#     label_path = os.path.join('data', 'train', 'labels', f'{image.split(".")[0]}.json')

#     # if label for image exists (if eyes are open or visible), update clases and coords
#     if os.path.exists(label_path):
#         with open(label_path, 'r') as f:
#             label = json.load(f)

#         # if we have just 1 label, we need to check which eye was labeled (one eye closed)
#         if len(label['shapes']) == 1:
#             if label['shapes'][0]['label'] == 'LeftEye':
#                 classes[0] = 1
#                 coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
#                 coords[1] = np.squeeze(label['shapes'][0]['points'])[1]
#             # if not left eye its the right eye
#             else:
#                 classes[1] = 1
#                 coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
#                 coords[3] = np.squeeze(label['shapes'][0]['points'])[1]

#         # if both eyes are labeled, we need to check which eye was labeled 1st or 2nd
#         if len(label['shapes']) == 2:
#             # check first if we labeled the left eye first or second and update coords/classes
#             if label['shapes'][0]['label'] =='LeftEye':
#                 classes[0] = 1
#                 coords[0] = np.squeeze(label['shapes'][0]['points'])[0]
#                 coords[1] = np.squeeze(label['shapes'][0]['points'])[1]
#             else:
#                 classes[0] = 1
#                 coords[0] = np.squeeze(label['shapes'][1]['points'])[0]
#                 coords[1] = np.squeeze(label['shapes'][1]['points'])[1]

#             # then check if we labeled the right eye first or second and update coords/classes
#             if label['shapes'][0]['label'] =='RightEye':
#                 classes[1] = 1
#                 coords[2] = np.squeeze(label['shapes'][0]['points'])[0]
#                 coords[3] = np.squeeze(label['shapes'][0]['points'])[1]
#             else:
#                 classes[1] = 1
#                 coords[2] = np.squeeze(label['shapes'][1]['points'])[0]
#                 coords[3] = np.squeeze(label['shapes'][1]['points'])[1]


#     # now apply the augmentation to the image
#     try:
#         # 30 augementations generated for each image
#         for x in range(30):
#             # set keypoints of eyes to include them in the augmentation process
#             keypoints = [(coords[:2]), (coords[2:])]
#             # perform augmentation
#             augmented = augmentor(image=img, keypoints=keypoints, class_labels=['LeftEye', 'RightEye'])
#             # save augmentated image with slightly different name
#             cv2.imwrite(os.path.join('data', 'train_aug', 'images', f'{image.split(".")[0]}.{x}.jpg'), augmented['image'])

#             # initiate annotations for augmentated data
#             annotation = {}
#             annotation['image'] = image
#             annotation['class'] = [0,0]
#             annotation['keypoints'] = [0,0,0,0]

#             # if label for image exists we change to the augmentated keypoints and classes
#             if os.path.exists(label_path):
#                 if len(augmented['keypoints']) > 0:
#                     for idx, cl in enumerate(augmented['class_labels']):
#                         if cl == 'LeftEye':
#                             annotation['class'][0] = 1
#                             annotation['keypoints'][0] = augmented['keypoints'][idx][0]
#                             annotation['keypoints'][1] = augmented['keypoints'][idx][1]
#                         if cl == 'RightEye':
#                             annotation['class'][1] = 1
#                             annotation['keypoints'][2] = augmented['keypoints'][idx][0]
#                             annotation['keypoints'][3] = augmented['keypoints'][idx][1]

#             # save label of augmentated images
#             with open(os.path.join('data', 'train_aug', 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
#                 json.dump(annotation, f)

#     except Exception as e:
#         print(e)

In [65]:
def create_directory(path):
    if not os.path.exists(path):
        os.makedirs(path)


def generate_augmented_pictures(data_category):
    # iterate over all images in data set
    for image in os.listdir(os.path.join("data", data_category, "images")):
        # i += 1 # remove later
        data_base = os.path.join("data", data_category)
        aug_base = os.path.join("data", f"{data_category}_aug")

        # Ensure base directories for augmented images and labels exist
        create_directory(os.path.join(aug_base, "images"))
        create_directory(os.path.join(aug_base, "labels"))
        # opens image as array
        img = cv2.imread(os.path.join("data", data_category, "images", image))

        # initiate classes (eyes visible or not, left and right)
        classes = [0, 0]

        # initiate coordiates of eyes if visible if not 0 (x-left, y-left, x-right, y-right)
        coords = [0, 0, 0, 0]

        # get corresponding label to current image (they have same file name)
        label_path = os.path.join(
            "data", data_category, "labels", f'{image.split(".")[0]}.json'
        )

        # if label for image exists (if eyes are open or visible), update clases and coords
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                label = json.load(f)

            # if we have just 1 label, we need to check which eye was labeled (one eye closed)
            if len(label["shapes"]) == 1:
                if label["shapes"][0]["label"] == "LeftEye":
                    classes[0] = 1
                    coords[0] = np.squeeze(label["shapes"][0]["points"])[0]
                    coords[1] = np.squeeze(label["shapes"][0]["points"])[1]
                # if not left eye its the right eye
                else:
                    classes[1] = 1
                    coords[2] = np.squeeze(label["shapes"][0]["points"])[0]
                    coords[3] = np.squeeze(label["shapes"][0]["points"])[1]

            # if both eyes are labeled, we need to check which eye was labeled 1st or 2nd
            if len(label["shapes"]) == 2:
                # check first if we labeled the left eye first or second and update coords/classes
                if label["shapes"][0]["label"] == "LeftEye":
                    classes[0] = 1
                    coords[0] = np.squeeze(label["shapes"][0]["points"])[0]
                    coords[1] = np.squeeze(label["shapes"][0]["points"])[1]
                else:
                    classes[0] = 1
                    coords[0] = np.squeeze(label["shapes"][1]["points"])[0]
                    coords[1] = np.squeeze(label["shapes"][1]["points"])[1]

                # then check if we labeled the right eye first or second and update coords/classes
                if label["shapes"][0]["label"] == "RightEye":
                    classes[1] = 1
                    coords[2] = np.squeeze(label["shapes"][0]["points"])[0]
                    coords[3] = np.squeeze(label["shapes"][0]["points"])[1]
                else:
                    classes[1] = 1
                    coords[2] = np.squeeze(label["shapes"][1]["points"])[0]
                    coords[3] = np.squeeze(label["shapes"][1]["points"])[1]

                # normalizing the data
                imageHeight = label["imageHeight"]
                imageWidth = label["imageWidth"]
                np.divide(coords, [imageHeight, imageWidth,
                          imageHeight, imageWidth])

        # now apply the augmentation to the image
        try:
            # 60 augementations generated for each image
            for x in range(60):
                # set keypoints of eyes to include them in the augmentation process
                keypoints = [(coords[:2]), (coords[2:])]
                # perform augmentation
                augmented = augmentor(
                    image=img, keypoints=keypoints, class_labels=[
                        "LeftEye", "RightEye"]
                )
                # save augmentated image with slightly different name
                cv2.imwrite(
                    os.path.join(
                        "data",
                        data_category + "_aug",
                        "images",
                        f'{image.split(".")[0]}.{x}.jpg',
                    ),
                    augmented["image"],
                )

                # initiate annotations for augmentated data
                annotation = {}
                annotation["image"] = image
                annotation["class"] = [0, 0]
                annotation["keypoints"] = [0, 0, 0, 0]

                # if label for image exists we change to the augmentated keypoints and classes
                if os.path.exists(label_path):
                    if len(augmented["keypoints"]) > 0:
                        for idx, cl in enumerate(augmented["class_labels"]):
                            if cl == "LeftEye":
                                annotation["class"][0] = 1
                                annotation["keypoints"][0] = augmented["keypoints"][
                                    idx
                                ][0]
                                annotation["keypoints"][1] = augmented["keypoints"][
                                    idx
                                ][1]
                            if cl == "RightEye":
                                annotation["class"][1] = 1
                                annotation["keypoints"][2] = augmented["keypoints"][
                                    idx
                                ][0]
                                annotation["keypoints"][3] = augmented["keypoints"][
                                    idx
                                ][1]

                annotation["keypoints"] = list(
                    np.divide(annotation["keypoints"], [size_in_pixels] * 4)
                )
                # save label of augmentated images
                with open(
                    os.path.join(
                        "data",
                        data_category + "_aug",
                        "labels",
                        f'{image.split(".")[0]}.{x}.json',
                    ),
                    "w",
                ) as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)

In [66]:
generate_augmented_pictures("train")
generate_augmented_pictures("val")
generate_augmented_pictures("test")